In [38]:
import subprocess
from pymol import cmd
from pdbfixer import PDBFixer
from Bio import PDB
import json
import tempfile
import requests
from openmm.app import PDBFile


non_standard = ['A23', 'A2L', 'A2M', 'A39', 'A3P', 'A44', 'A5O', 'A6A', 'A7E', 'A9Z', 'ADI', 'ADP', 'AET', 'AMD', 'AMO', 'AP7', 'AVC', 'MA6', 'MAD', 'MGQ', 'MIA', 'MTU', 'M7A', '26A', '2MA', '6IA', '6MA', '6MC', '6MP', '6MT', '6MZ', '6NW', 'F3N', 'N79', 'RIA', 'V3L', 'ZAD', '31H', '31M', '7AT', 'O2Z', 'SRA', '00A', '45A', '8AN', 'LCA', 'P5P', 'PPU', 'PR5', 'PU', 'T6A', 'TBN', 'TXD', 'TXP', '12A', '1MA', '5FA', 'A6G', 'E6G', 'E7G', 'EQ4', 'IG', 'IMP', 'M2G', 'MGT', 'MGV', 'MHG', 'QUO', 'YG', 'YYG', '23G', '2EG', '2MG', '2SG', 'B8K', 'B8W', 'B9B', 'BGH', 'N6G', 'RFJ', 'ZGU', '7MG', 'CG1', 'G1G', 'G25', 'G2L', 'G46', 'G48', 'G7M', 'GAO', 'GDO', 'GDP', 'GH3', 'GNG', 'GOM', 'GRB', 'GTP', 'KAG', 'KAK', 'O2G', 'OMG', '8AA', '8OS', 'LG', 'PGP', 'P7G', 'TPG', 'TG', 'XTS', '102', '18M', '1MG', 'A5M', 'A6C',
                'E3C', 'IC', 'M4C', 'M5M', '6OO', 'B8Q', 'B8T', 'B9H', 'JMH', 'N5M', 'RPC', 'RSP', 'RSQ', 'ZBC', 'ZCY', '73W', 'C25', 'C2L', 'C31', 'C43', 'C5L', 'CBV', 'CCC', 'CH', 'CSF', 'OMC', 'S4C', '4OC', 'LC', 'LHH', 'LV2', 'PMT', 'TC', '10C', '1SC', '5HM', '5IC', '5MC', 'A6U', 'IU', 'I4U', 'MEP', 'MNU', 'U25', 'U2L', 'U2P', 'U31', 'U34', 'U36', 'U37', 'U8U', 'UAR', 'UBB', 'UBD', 'UD5', 'UPV', 'UR3', 'URD', 'US5', 'UZR', 'UMO', 'U23', '2AU', '2MU', '2OM', 'B8H', 'FHU', 'FNU', 'F2T', 'RUS', 'ZBU', '3AU', '3ME', '3MU', '3TD', '70U', '75B', 'CNU', 'OMU', 'ONE', 'S4U', 'SSU', 'SUR', '4SU', '85Y', 'DHU', 'H2U', 'LHU', 'PSU', 'PYO', 'P4U', 'T31', '125', '126', '127', '1RN', '5BU', '5FU', '5MU', '9QV', '5GP', "GTA", "F86", "I", "ATP", "DOC", "GMP", "GP3", "CSL", "M7G", "C5P", "RY", "LCC", "AMP", "G5J", "CFL", "UFT"]
parser = dict()
res = requests.get(
    'https://rna.bgsu.edu/modified/modified_to_change_data.json')
p_file = ''
with tempfile.NamedTemporaryFile(suffix='.json', delete=False) as par:
    par.write(res.content)
    p_file = par.name
with open(p_file, 'r') as par_f:
    parser = json.load(par_f)
parser['DU'] = {'standard_base': ['U']}
parser['N'] = {'standard_base': ['*']}
parser['ADP'] = {'standard_base': ['A']}
parser['UFT'] = {'standard_base': ['U']}


def fix(path,pz):
    class REMOVEOP11OP2FirstSelect(PDB.Select):
        def __init__(self):
            pass

        def accept_atom(self, atom):
            if atom.get_parent().get_id()[1] == 1:
                return atom.get_name() not in ["OP1", "OP2","P"]
            else:
                return True
    fixer = PDBFixer(filename=path)
    fixer.findNonstandardResidues()
    fixer.missingResidues = {}
    fixer.nonstandardResidues = list(
        [
            (i, parser[i.name]["standard_base"][0])
            for i in fixer.topology.residues()
            if i.name in non_standard
        ]
    )
    fixer.replaceNonstandardResidues()
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    PDBFile.writeFile(fixer.topology, fixer.positions, open(path, "w"))
    ref = PDB.PDBParser(QUIET=True).get_structure("str", path)[0]
    ref = ref[[i.id for i in ref][0]]
    task = subprocess.Popen(
        f"rna_pdb_tools.py --get-rnapuzzle-ready {path} --inplace".split(),
        stderr=subprocess.PIPE,
        stdout=subprocess.PIPE,
    )
    r, e = task.communicate()
    io = PDB.PDBIO()
    io.set_structure(ref)
    io.save(
        path,
        REMOVEOP11OP2FirstSelect(),
    )

    # if task.returncode != 0:
    #     raise ValueError(f"rna_pdb_tools failed {path}" + e.decode("utf-8"))

In [ ]:
import pandas as pd
import os

from tqdm import tqdm

result = pd.DataFrame({"PZ": [], "Lab": [], "Num": [], "RMSD": []})
puzzle_set = pd.read_csv(
    "/home/adamczykb/rnaquanet/data/00_reference/rnapuzzle/rnapuzzle_description.csv"
)
path_to_dataset = "/home/adamczykb/rnaquanet/data/00_reference/rnapuzzle/data"
for row, pzset in puzzle_set.iterrows():
    path_to_subset = os.path.join(path_to_dataset, "PZ" + str(pzset["rna_puzzle_no"]))
    solutions = [
        i
        for i in os.listdir(os.path.join(path_to_subset, "test"))
        if "solution" in i and "~" not in i
    ]
    for sol in solutions:
        fix(f"{os.path.join(path_to_subset, 'test',sol)}", str(pzset["rna_puzzle_no"]))
    print(f"Processing puzzle {pzset['rna_puzzle_no']} with {len(solutions)} solutions")
    for srow, entry in pd.read_csv(os.path.join(path_to_subset, "test.csv")).iterrows():

        values = []
        errors = []
        try:
            fix(
                f"{os.path.join(path_to_subset, 'test')}/{'PZ'+str(pzset['rna_puzzle_no'])}_{entry['Lab']}_{entry['Num']}.pdb",
                str(pzset["rna_puzzle_no"]),
            )
            for sol in solutions:

                cmd.reinitialize()

                cmd.load(
                    f"{os.path.join(path_to_subset, 'test')}/{'PZ'+str(pzset['rna_puzzle_no'])}_{entry['Lab']}_{entry['Num']}.pdb",
                    object="_model",
                )
                cmd.load(os.path.join(path_to_subset, "test", sol), object="reference")
                cmd.hide("everything")
                cmd.show("cartoon")
                cmd.color("white")
                try:
                    values.append(cmd.pair_fit(f"_model", "reference"))
                except Exception as e:
                    errors.append(
                        f"Error in pair_fit for {sol} {os.path.join(path_to_subset, 'test')}/{'PZ'+str(pzset['rna_puzzle_no'])}_{entry['Lab']}_{entry['Num']}.pdb {os.path.join(path_to_subset, 'test', sol)}: {e}"
                    )
                cmd.delete("_model")
                cmd.delete("reference")
            if len(values) == 0:
                for error in errors:
                    print(f" | {error}")
                continue
            result = pd.concat(
                [
                    pd.DataFrame(
                        [
                            [
                                pzset["rna_puzzle_no"],
                                entry["Lab"],
                                entry["Num"],
                                min(values),
                            ]
                        ],
                        columns=result.columns,
                    ),
                    result,
                ],
                ignore_index=True,
            )
        except Exception as e:
            print(
                f"Error in processing {os.path.join(path_to_subset, 'test')}/{'PZ'+str(pzset['rna_puzzle_no'])}_{entry['Lab']}_{entry['Num']}.pdb: {e}"
            )
            continue

Processing puzzle 38 with 1 solutions
 ExecutiveRMSPairs: RMSD =    8.474 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =    9.875 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   10.147 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   11.329 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   11.447 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   11.569 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   12.573 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   12.866 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   12.983 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   13.997 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   14.391 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   14.366 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   15.269 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   15.552 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   17.956 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =   20.132 (1182 to 1182 atoms)
 ExecutiveRMSPairs: RMSD =    8.492 (1182 to 1182 